# 06 SemanticKernel | 05 Planner

## Planner

With Planner Semantic Kernel provides concept where a call to a LLM from the GPT family is asked to create an execution plan for a given task based on functionality provided to Semantic Kernel by Plugins.

Additional information: 
- [Overview Memory](./README.md)
- [What is Semantic Kernel](https://learn.microsoft.com/en-us/semantic-kernel/overview/)
- [Semantic Kernel Planner](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/planners/?tabs=Csharp)

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_DemoEnvironment/01_Environment.ipynb)) 

## Step 1: Create Semantic Kernel instance

An instance of Semantic Kernel can be created using the KernelBuilder object provided by the Semantic Kernel SDK. It acts as the centralized point for all .NET functionality that want to interact with Semantic Kernel functionality or concepts.

It abstracts e.g., models from the OpenAI GPT family and can communicate with Azure OpenAI LLM instances as well as LLMs deployed on OpenAI. In the sample we use models deployed on Azure.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta1"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using DotNetEnv;
using Microsoft.SemanticKernel;

//configuration file is created during environment creation
//if you skipped the deployment just remove the code and provide values from your deployment
static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI_APIKEY") ?? "SKIT_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI_ENDPOINT") ?? "SKIT_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("SKIT_CHATCOMPLETION_DEPLOYMENTNAME") ?? "SKIT_CHATCOMPLETION_DEPLOYMENTNAME not found";

IKernel kernel = new KernelBuilder()
    .WithAzureChatCompletionService(
        endpoint: oAiEndpoint, 
        apiKey: oAiApiKey,
        deploymentName: chatCompletionDeploymentName)
    .Build();

Console.WriteLine($"Semantic Kernel initialized...");


#### Expected output:

```
Installed Packages
DotNetEnv, 2.5.0
Microsoft.SemanticKernel, 1.0.0-beta1

Semantic Kernel initialized...
```

## Step 2: Function definition (native & semantic from file)

Two functions are defined and will be registered with Semantic Kernel. One function (native function) retrieves content from a local file. The second function (file based semantic function) performs a summarization of provided text.

In [ ]:
using Microsoft.SemanticKernel.Orchestration;
using System.ComponentModel;
using System.IO;

public class NativeFunctions {

    [SKFunction, Description("Retrieve content from local file")]
    public async Task<string> RetrieveLocalFile(string fileName, int maxSize = 5000) 
    {
        string content = await File.ReadAllTextAsync(fileName); 
        if (content.Length <= maxSize) return content;
        return content.Substring(0, maxSize);

    }
}

//Import native function
string plugInName = "NativeFunction";
string functionName = "RetrieveLocalFile";

NativeFunctions nativeFunctions = new NativeFunctions();
kernel.ImportFunctions(nativeFunctions, plugInName);

Console.WriteLine($"Inline function {functionName} defined and imported...");

string pluginsDirectory = Path.Combine(Directory.GetCurrentDirectory(), "PluginLibrary");
plugInName = "CreateSummary";
kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, plugInName);

Console.WriteLine($"Semantic functions imported from directory '{pluginsDirectory}'...");

#### Expected output:

```
Inline function RetrieveLocalFile defined and imported...
Semantic functions imported from directory 'c:\Sourcen\GitHubProjects\OpenAI.StarterKit\docs\07_SemanticKernel\PluginLibrary'...
```

## Step 4: Create & execute Planner

An ask *"Read content from a local file and summarize the content"* is defined and a new `SequentialPlanner` instance is created. 
`SequentialPlanner` provides a function `CreatePlanAsync()` which takes the defined ask, creates based on registered functions (by using the LLM registered with Semantic Kernel) an execution Plan. This plan includes all functions which should be called including sequence and parameters. To execute the plan the kernel function `RunAsync()` is called.

In [ ]:
using Microsoft.SemanticKernel.Planners;
using Microsoft.SemanticKernel.Orchestration;

string actionDefinition = "Read content from a local file and summarize the content.";

SequentialPlanner sequentialPlanner = new SequentialPlanner(kernel);      

string assetsFolder = @"../../assets";
string fileName = Path.Combine(assetsFolder,"docs","07_SemanticKernel", "aci_documentation.txt");

ContextVariables contextVariables = new ContextVariables();
contextVariables.Add("fileName", fileName);

var customPlan = await sequentialPlanner.CreatePlanAsync(actionDefinition);

KernelResult kernelResult = await kernel.RunAsync(contextVariables, customPlan);

Console.WriteLine($"Summarization: {kernelResult.GetValue<string>()}");


#### Expected output:

```
Summarization: To configure a GitHub Action to create ...
```

## Next steps

- [Add logging functionality to Semantic Kernel ](./06_Logs.ipynb)

